create databricks secret : 
databricks secrets create-scope ticker
databricks secrets list-scopes
databricks secrets delete-scope ticker
Create a secret

databricks secrets put-secret --json '{
  "scope": "ticker",
  "key": "financialmodelprep_token",
  "string_value": "apikey"
}'


databricks secrets put-secret --json '{
  "scope": "ticker",
  "key": "access_key",
  "string_value": "access_key"
}'


databricks secrets put-secret --json '{
  "scope": "ticker",
  "key": "secret_key",
  "string_value": "secret_key"
}'
databricks secrets put-secret --json '{
  "scope": "ticker",
  "key": "session_key",
  "string_value": "session_key"
}'

In [0]:
import requests
import json
from datetime import datetime
dbutils.widgets.text("run_date", datetime.now().strftime("%Y-%m-%d"),"Run date")
run_date = dbutils.widgets.get("run_date")

APIKey = dbutils.secrets.get(scope = "ticker", key = "financialmodelprep_token")
BaseUrl = "https://financialmodelingprep.com"
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "JPM", "V", "JNJ","PG"]
# TICKERS = ["AAPL"]


In [0]:
def process_ticker(ticker,run_date):
    Income_statement_Query = f"{BaseUrl}/stable/income-statement?symbol={ticker}&limit=5&period=quarter&apikey={APIKey}"
    Balance_sheet_statement_Query = f"{BaseUrl}/stable/balance-sheet-statement?symbol={ticker}&limit=5&period=quarter&apikey={APIKey}"
    cashflow_statement_Query = f"{BaseUrl}/stable/cash-flow-statement?symbol={ticker}&limit=5&period=quarter&apikey={APIKey}"

    urls = [Income_statement_Query,Balance_sheet_statement_Query,cashflow_statement_Query]
  
    filename = datetime.now().strftime("%Y%m%d_%H%M%S")
    files = []
    try:
        for Query in urls:
            response = requests.get(Query, timeout=10)
            response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx) 
            data = response.json()
            label = Query.split('/')[4].split('?')[0]

            # CRITICAL Partitioning for Spark: /source=fmp/ticker=AAPL/date=2023-10-25/
            file_name = f"landing/source=fmp/ticker={ticker}/date={run_date}/statement={label}/{filename}.json"
            
            json_data = json.dumps(data, indent=2)
            files.append({"file_name": file_name, "json_data": json_data,"status" : response.status_code})
        return files
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")
        file_name = f"landing/source=fmp/audit_failures/ticker={ticker}/date={run_date}/{filename}.json"
        status_code =  response.status_code if 'response' in locals() else 500
        error_log = {
            "error": str(e),
            "ticker": ticker,
            "status_code": status_code
        }
        return [{"file_name": file_name, "json_data": json.dumps(error_log,indent=2), "status" : status_code}]

        


In [0]:
flat_list = []
for ticker in TICKERS:
    flat_list.extend(process_ticker(ticker,run_date))

print(f"data length {len(flat_list)} and symbols found {set([ticker['file_name'] if 'file_name' in ticker else None for ticker in flat_list])}")


To connect and store data fro mdatabricks to s3
we need IAM role in AWS and storage crednetial in databricks created in unity catalog
i have no access to create a role or group of policies in AWS that said  i will use less secure method and coomunciate directy with s3
unfortunally as i am using aws lab educate , i can do nothing iwth i am i will teh lab secrects and key to communciate with s3


In [0]:
import json
import boto3

ACCESS_KEY = dbutils.secrets.get(scope = "ticker", key = "access_key")
SECRET_KEY = dbutils.secrets.get(scope = "ticker", key = "secret_key")
SESSION_TOKEN = dbutils.secrets.get(scope = "ticker", key = "session_key")

# "taskKey" must match the NAME of the task in your Job workflow (e.g., "Init_Auth")
temp_ak = dbutils.jobs.taskValues.get(taskKey="Init_Auth", key="temp_ak", debugValue="debug-key")
temp_sk = dbutils.jobs.taskValues.get(taskKey="Init_Auth", key="temp_sk", debugValue="debug-secret")
temp_token = dbutils.jobs.taskValues.get(taskKey="Init_Auth", key="temp_token", debugValue="debug-token")

# 3. Initialize S3 client or resource
# s3 = boto3.client(
#     's3',
#     aws_access_key_id=temp_ak,
#     aws_secret_access_key=temp_sk,
#     aws_session_token=temp_token
# )

s3 = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN
)



for ticker in flat_list:
    print(ticker['file_name'])

    # 4. Specify your bucket name and the desired file name (key) in S3
    bucket_name = 'arn:aws:s3:us-east-1:180250667274:accesspoint/accesspoint-to-data'
    s3_object_key = ticker['file_name']

    # 5. Upload the JSON string
    s3.put_object(
    Bucket=bucket_name,
    Key=s3_object_key,
    Body=ticker['json_data']
)
  

    print(f"Successfully uploaded JSON data to s3://{bucket_name}/{s3_object_key}")

The Verification: Standards Check


Resilience (Fail Safely): PASS. You use try/except and route errors to audit_failures instead of crashing. 



Observability: PASS. You capture the exact error message and status code in the DLQ JSON. 


Disaster Recovery: PASS. You correctly added the widget run_date, allowing you to run "Protocol A: The Surgical Backfill"  by passing a specific date.

Data Integrity: PASS. You removed the second json.dumps(), preventing the "Double String" corruption. The data will arrive in Bronze as a proper Struct.